In [3]:
import json
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import mean_squared_error
from pathlib import Path

In [4]:
data_path = Path("LightGBM_examples/regression")

In [6]:
print('Load data...')
df_train = pd.read_csv(data_path/'regression.train', header=None, sep='\t')
df_test = pd.read_csv(data_path/'regression.test', header=None, sep='\t')

Load data...


In [7]:
y_train = df_train[0].values
y_test = df_test[0].values
X_train = df_train.drop(0, axis=1).values
X_test = df_test.drop(0, axis=1).values

In [8]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [9]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'auc'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

In [10]:
print('Start training...')
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

print('Save model...')
# save model to file
gbm.save_model('model.txt')

Start training...
[1]	valid_0's l2: 0.243898	valid_0's auc: 0.721096
Training until validation scores don't improve for 5 rounds
[2]	valid_0's l2: 0.240605	valid_0's auc: 0.732932
[3]	valid_0's l2: 0.236472	valid_0's auc: 0.773583
[4]	valid_0's l2: 0.232586	valid_0's auc: 0.781089
[5]	valid_0's l2: 0.22865	valid_0's auc: 0.787902
[6]	valid_0's l2: 0.226187	valid_0's auc: 0.780565
[7]	valid_0's l2: 0.223738	valid_0's auc: 0.786571
[8]	valid_0's l2: 0.221012	valid_0's auc: 0.784918
[9]	valid_0's l2: 0.218429	valid_0's auc: 0.784878
[10]	valid_0's l2: 0.215505	valid_0's auc: 0.788917
[11]	valid_0's l2: 0.213027	valid_0's auc: 0.791586
[12]	valid_0's l2: 0.210809	valid_0's auc: 0.793884
[13]	valid_0's l2: 0.208612	valid_0's auc: 0.796174
[14]	valid_0's l2: 0.207468	valid_0's auc: 0.793731
[15]	valid_0's l2: 0.206009	valid_0's auc: 0.794247
[16]	valid_0's l2: 0.20465	valid_0's auc: 0.793715
[17]	valid_0's l2: 0.202489	valid_0's auc: 0.797085
[18]	valid_0's l2: 0.200668	valid_0's auc: 0.8026

In [11]:
print('Start predicting...')
# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
# eval
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

Start predicting...
The rmse of prediction is: 0.44512434910807497
